In [1]:
import requests

In [2]:
url = 'https://search.jd.com/s_new.php?keyword=iphone13&enc=utf-8&wq=iphone13&page=1'

In [3]:
headers = {
    'authority': 'search.jd.com',
    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
    'accept': '*/*',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://search.jd.com/Search?keyword=iphone13&enc=utf-8&suggest=1.def.0.base&wq=iph&pvid=647ad05359304d7f9e3ea455d85f804b',
    'accept-language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
}

params = (
    ('keyword', 'iphone13'),
    ('suggest', '1.def.0.base'),
    ('wq', 'iphone13'),
    ('pvid', '647ad05359304d7f9e3ea455d85f804b'),
    ('page', '2'),
    ('s', '26'),
    ('scrolling', 'y'),
    ('log_id', '1633503285671.1827'),
    ('tpl', '1_M'),
    ('isList', '0'),
)


In [4]:
response = requests.get(url=url, headers=headers, params=params)
response.encoding="utf-8"

In [5]:
from lxml import etree

In [6]:
html = etree.HTML(response.text)

In [7]:
#价格
price_list = html.xpath('//div[@class="gl-i-wrap"]/div[@class="p-price"]/strong/i/text()')

In [8]:
#名字
name_list = []

name_list_etree = html.xpath('//div[@class="p-name p-name-type-2"]/a/em')

for name in name_list_etree:
    name_list.append(str(name.xpath('string(.)')).replace('\n',''))

In [9]:
#店铺名
shopname_list = []

shopname_list_etree = html.xpath('//span[@class="J_im_icon"]/a')

for shopname in shopname_list_etree:
    shopname_list.append(shopname.text)

In [10]:
#评论数
import json
list_id = html.xpath('//li[@class="gl-item"]/@data-sku')#获取商品id

url_id = 'https://club.jd.com/comment/productCommentSummaries.action?referenceIds='

for pid in list_id:#得到评论数的url
    url_id += pid + ","

comment = requests.get(url=url_id,headers=headers, params=params)
comment_list = []

for comment in comment.json()["CommentsCount"]:
    comment_list.append(comment["CommentCountStr"])

In [11]:
import pandas as pd

dataframe = pd.DataFrame({
    '价格':price_list,
    '名字':name_list,
    '店铺名':shopname_list,
    '评论数':comment_list,
})

dataframe.to_csv("08_京东_request.csv",index=False)